In [14]:
#imports
import numpy as np
import pandas as pd
import math
import pickle

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import PolynomialFeatures,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

#import training dataset
train_df = pd.read_csv("samletdata.csv", sep = ';')

#see the columns in our data
train_df.info()

# take a look at the head of the dataset
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99187 entries, 0 to 99186
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    99187 non-null  int64 
 1   model         99187 non-null  object
 2   year          99187 non-null  int64 
 3   price         99187 non-null  int64 
 4   transmission  99187 non-null  object
 5   mileage       99187 non-null  int64 
 6   fuelType      99187 non-null  object
 7   tax           99187 non-null  int64 
 8   mpg           99187 non-null  object
 9   engineSize    99187 non-null  object
 10  brand         99187 non-null  object
dtypes: int64(5), object(6)
memory usage: 8.3+ MB


,Unnamed: 0,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand
0,0,A1,2017,12500,Manual,15735,Petrol,150,"55,4","1,4",audi
1,1,A6,2016,16500,Automatic,36203,Diesel,20,"64,2","2,0",audi
2,2,A1,2016,11000,Manual,29946,Petrol,30,"55,4","1,4",audi
3,3,A4,2017,16800,Automatic,25952,Diesel,145,"67,3","2,0",audi
4,4,A3,2019,17300,Manual,1998,Petrol,145,"49,6","1,0",audi


In [20]:
features = ['model', 'year', 'transmission', 'mileage','fuelType', 'mpg', 'engineSize', 'brand']

x = train_df[features]
enc = OneHotEncoder(handle_unknown='ignore')
x = enc.fit_transform(x)

y = train_df['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.3)

In [23]:
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

print('Training score: {}'.format(lr_model.score(x_train, y_train)))
print('Test score: {}'.format(lr_model.score(x_test, y_test)))

y_pred = lr_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)

print('RMSE: {}'.format(rmse))

import pickle
pickle.dump(lr_model, open("lr_modeldinmor.🤓", "ab"))

Training score: 0.9537519340466061
Test score: 0.8671850138901346
RMSE: 3596.587037967226


In [6]:
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("samletdata.csv", sep = ';')
print(df)

       Unnamed: 0 model  year  price transmission  mileage fuelType  tax   
0               0    A1  2017  12500       Manual    15735   Petrol  150  \
1               1    A6  2016  16500    Automatic    36203   Diesel   20   
2               2    A1  2016  11000       Manual    29946   Petrol   30   
3               3    A4  2017  16800    Automatic    25952   Diesel  145   
4               4    A3  2019  17300       Manual     1998   Petrol  145   
...           ...   ...   ...    ...          ...      ...      ...  ...   
99182       15152   Eos  2012   5990       Manual    74000   Diesel  125   
99183       15153   Fox  2008   1799       Manual    88102   Petrol  145   
99184       15154   Fox  2009   1590       Manual    70000   Petrol  200   
99185       15155   Fox  2006   1250       Manual    82704   Petrol  150   
99186       15156   Fox  2007   2295       Manual    74000   Petrol  145   

        mpg engineSize brand  
0      55,4        1,4  audi  
1      64,2        2,0  a

In [9]:

train, test = train_test_split(df, test_size=0.2, shuffle=True)
features = ['model', 'year', 'transmission', 'mileage','fuelType', 'tax', 'mpg', 'engineSize', 'brand']

X = train[features]
y = train['price']

X_test = test[features]
y_test = test['price']

dtree = DecisionTreeClassifier()
print(X,y)
dtree = dtree.fit(X, y)

tree.plot_tree(dtree, feature_names=features) 

            model  year transmission  mileage fuelType  tax   mpg engineSize   
14614    4 Series  2020    Semi-Auto     1000   Petrol  145  48,7        2,0  \
46757     E Class  2019    Semi-Auto     7651   Petrol  145  39,8        2,0   
12926    3 Series  2019    Semi-Auto     8349   Petrol  145  43,5        2,0   
86270        Golf  2019    Automatic     4173   Petrol  145  44,8        1,0   
40509        Kona  2019       Manual     8550   Petrol  145  44,1        1,0   
...           ...   ...          ...      ...      ...  ...   ...        ...   
29219       C-MAX  2017       Manual    20357   Petrol  150  55,4        1,0   
51261   GLA Class  2015    Semi-Auto    47000   Diesel  125  56,5        2,1   
67621        Aygo  2015       Manual    33248   Petrol    0  69,0        1,0   
3664           TT  2015       Manual    31321   Petrol  150  47,9        2,0   
64209        RAV4  2017    Automatic    65452   Hybrid   20  57,6        2,5   

          brand  
14614       bmw  
467

ValueError: could not convert string to float: ' 4 Series'